In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Importing Information

In [61]:
df = pd.read_csv("data/Food_Inspections.csv")

df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,1068208,CHINA COURT RESTAURANT,CHINA COURT RESTAURANT,2141795.0,Restaurant,Risk 1 (High),1146 N MILWAUKEE AVE,CHICAGO,IL,60642.0,03/14/2012,License Re-Inspection,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.902462,-87.665306,"(41.902462266949634, -87.66530609467256)"
1,1072213,CUDDLE CARE,CUDDLE CARE,1622366.0,Daycare Above and Under 2 Years,Risk 1 (High),4800 S LAKE PARK AVE,CHICAGO,IL,60615.0,10/22/2012,Canvass,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.807922,-87.590693,"(41.80792179224785, -87.5906931090992)"
2,1072214,CUDDLE CARE,CUDDLE CARE,1622365.0,Daycare Above and Under 2 Years,Risk 1 (High),4800 S LAKE PARK AVE,CHICAGO,IL,60615.0,10/22/2012,Canvass,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.807922,-87.590693,"(41.80792179224785, -87.5906931090992)"
3,1072228,SHARKS FISH & CHICKEN,SHARKS FISH & CHICKEN,2069562.0,Restaurant,Risk 2 (Medium),101 E 51ST ST,CHICAGO,IL,60615.0,10/26/2012,Short Form Complaint,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.801892,-87.622566,"(41.80189221533366, -87.62256558837282)"
4,1072252,SALAAM RESTAURANT AND BAKERY,SALAAM RESTAURANT AND BAKERY,2141327.0,Restaurant,Risk 1 (High),700-706 W 79TH ST,CHICAGO,IL,60620.0,01/24/2013,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.750787,-87.641667,"(41.750787498480555, -87.64166664542023)"


In [62]:
df["Inspection Date"] = pd.to_datetime(df["Inspection Date"], errors = 'coerce')

# Lets filter by the last 5 years

latest_date = df["Inspection Date"].max()
cutoff = latest_date - pd.DateOffset(years = 5)
df_recent = df[df["Inspection Date"] >= cutoff]

df_recent.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
343,2555052,CITGO,CITGO,2575301.0,Grocery Store,Risk 2 (Medium),8601-8615 S HALSTED ST,CHICAGO,IL,60620.0,2022-04-29,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.737974,-87.643525,"(41.737973780282466, -87.64352541078881)"
746,2555152,F & K TRADING INC.,TOP MIDDLE EASTERN RESTAURANT,2845330.0,Restaurant,Risk 1 (High),5750 N CALIFORNIA AVE,CHICAGO,IL,60659.0,2022-05-03,License,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.986476,-87.699415,"(41.9864759436678, -87.69941539025004)"
5016,2401719,HOTEL LINCOLN,J. PARKER/HOTEL LINCOLN/ELAINE'S,2442928.0,Restaurant,Risk 1 (High),1816 N CLARK ST,CHICAGO,IL,60614.0,2020-09-28,Canvass,Fail,25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...,41.915250,-87.634293,"(41.91524998684686, -87.63429326814013)"
5399,2453666,FULL SHILLING,FULL SHILLING,32813.0,Restaurant,Risk 1 (High),3724-3726 N CLARK ST,CHICAGO,IL,60613.0,2020-10-21,Canvass,No Entry,NaN,41.949706,-87.658701,"(41.94970619451691, -87.65870141348526)"
5451,2484917,"THE GRAND CHILD CARE CENTER, INC.","THE GRAND CHILD CARE CENTER, INC.",2220002.0,Children's Services Facility,Risk 1 (High),5945 W GRAND AVE,CHICAGO,IL,60639.0,2021-02-19,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.920326,-87.774947,"(41.920325606712524, -87.77494740927297)"


In [63]:
df_clean = df_recent.copy()

# Create a Passed column from the results column, a passing grade will give a true value in the passed column
df_clean['Passed'] = df_clean['Results'].str.lower().str.contains("pass")

df_clean.dropna(subset = ['Zip', 'Latitude', 'Longitude'], inplace = True)

# Here we convert the float to int, then that same int into a string
df_clean['Zip'] = df_clean['Zip'].astype(int).astype(str)

# With this, we will separate the violation code with the rest of the violation for example separate the number 10. from "10. ADEQUATE HANDWASHING SINKS..."
df_clean['Primary Violation Code'] = (
    df_clean['Violations']
    .dropna()
    .str.extract(r'(\d+)\.')
    .astype(float)
)

df_clean['License #'] = df_clean['License #'].astype('Int64').astype(str)

# After all this data cleaning lets take a peek at our data set now
df_clean.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location,Passed,Primary Violation Code
343,2555052,CITGO,CITGO,2575301,Grocery Store,Risk 2 (Medium),8601-8615 S HALSTED ST,CHICAGO,IL,60620,2022-04-29,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.737974,-87.643525,"(41.737973780282466, -87.64352541078881)",False,10.0
746,2555152,F & K TRADING INC.,TOP MIDDLE EASTERN RESTAURANT,2845330,Restaurant,Risk 1 (High),5750 N CALIFORNIA AVE,CHICAGO,IL,60659,2022-05-03,License,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.986476,-87.699415,"(41.9864759436678, -87.69941539025004)",False,1.0
5016,2401719,HOTEL LINCOLN,J. PARKER/HOTEL LINCOLN/ELAINE'S,2442928,Restaurant,Risk 1 (High),1816 N CLARK ST,CHICAGO,IL,60614,2020-09-28,Canvass,Fail,25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...,41.915250,-87.634293,"(41.91524998684686, -87.63429326814013)",False,25.0
5399,2453666,FULL SHILLING,FULL SHILLING,32813,Restaurant,Risk 1 (High),3724-3726 N CLARK ST,CHICAGO,IL,60613,2020-10-21,Canvass,No Entry,NaN,41.949706,-87.658701,"(41.94970619451691, -87.65870141348526)",False,NaN
5451,2484917,"THE GRAND CHILD CARE CENTER, INC.","THE GRAND CHILD CARE CENTER, INC.",2220002,Children's Services Facility,Risk 1 (High),5945 W GRAND AVE,CHICAGO,IL,60639,2021-02-19,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.920326,-87.774947,"(41.920325606712524, -87.77494740927297)",True,36.0


In [64]:
# Lets quickly see how many tests passed and how many failed
print(df_clean['Passed'].value_counts(normalize = True) * 100)


Passed
True     64.298397
False    35.701603
Name: proportion, dtype: float64


In [65]:
zipcode_failrate = (
    df_clean.groupby('Zip')['Passed']
    .apply(lambda x: 100 * (1 - x.mean()))
    .sort_values(ascending = False)
)

# Top 10 zipcodes that failed inspections
zipcode_failrate.head(10)

Zip
60827    45.205479
60651    44.739022
60620    43.812709
60624    42.450766
60639    42.345679
60617    41.933671
60625    41.323648
60621    41.018388
60636    40.543260
60644    40.152339
Name: Passed, dtype: float64

In [66]:
# Another interesting point is to see how many inspections happen on a month basis on average.
average_inspections = df_clean.set_index('Inspection Date').resample('ME')['Passed'].mean().count()

print(f"There are roughtly {average_inspections} inspections per month")

There are roughtly 61 inspections per month


In [67]:
# Here we will explore what type of establishments tend to fail the most
failed_tests = df_clean[df_clean['Passed'] == False]

failed_tests['Facility Type'].value_counts().head(10)

Facility Type
Restaurant                         21711
Grocery Store                       3281
School                              1528
Children's Services Facility         757
Bakery                               444
Daycare Above and Under 2 Years      398
Long Term Care                       271
Catering                             220
Daycare (2 - 6 Years)                211
Liquor                               168
Name: count, dtype: int64

In [68]:
df_clean.to_csv("clean_food_inspections.csv", index = False)